# CIFAR-10: Adversarial Training and Defenses

## Imports and CIFAR-10 loading

In [1]:
# For plotting
import numpy as np
import torch
import torch.nn as nn

# Nice loading bars
from tqdm.notebook import tnrange, tqdm

# DNN used
import models.resnet as resnet

# Test the loaded model
import utils.clean_test as clean_test

In [2]:
# Define the `device` PyTorch will be running on, please hope it is CUDA
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Notebook will use PyTorch Device: " + device.upper())

Notebook will use PyTorch Device: CUDA


## Clean Datasets

In [3]:
# Get the data loaders (assume we do no validation)
import utils.dataloaders as dataloaders

DATA_ROOT = "./datasets/CIFAR10"

trainSetLoader, _, testSetLoader = dataloaders.get_CIFAR10_data_loaders(
    DATA_ROOT,
    trainSetSize=50000,
    validationSetSize=0,
    batchSize=128,
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## Adversarial Training

In [4]:
import attacks.fgsm as fgsm
import attacks.pgd as pgd

attacks = {}
attacks["FGSM"] = fgsm.fgsm_attack
attacks["PGD"] = pgd.pgd_attack

import utils.attacking as attacking

# For printing outcomes
import utils.printing as printing

In [5]:
# Adversarial examples should be typically generated when model parameters are not
# changing i.e. model parameters are frozen. This step may not be required for very
# simple linear models, but is a must for models using components such as dropout
# or batch normalization.


# Note: to speed up training, using this https://arxiv.org/abs/2001.03994 variant
def get_adversarially_trained_model(attack, **kwargs):
    # For 200 epochs, use 150, 200 as the epochs tresholds
    # Also learning rates are 0.05, 0.01, 0.001

    # Helps adjust learning rate for better results
    def adjust_learning_rate(optimizer, epoch, learning_rate):
        actual_learning_rate = learning_rate
        if epoch >= 150:
            actual_learning_rate = 0.01
        if epoch >= 200:
            actual_learning_rate = 0.001
        for param_group in optimizer.param_groups:
            param_group["lr"] = actual_learning_rate

    # Various training parameters
    epochs = 30
    learning_rate = 0.1

    # Network parameters
    loss_function = nn.CrossEntropyLoss()
    model = resnet.ResNet18()
    model = model.to(device)
    model = nn.DataParallel(model)

    # Consider using ADAM here as another gradient descent algorithm
    optimizer = torch.optim.SGD(
        model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002
    )

    # Check if using epsilon
    if "epsilon" in kwargs:
        epsilon = kwargs["epsilon"]
    else:
        epsilon = None

    # Check if using alpha
    if "alpha" in kwargs:
        alpha = kwargs["alpha"]
    else:
        alpha = None

    # Get iterations
    if "iterations" in kwargs:
        iterations = kwargs["iterations"]
    else:
        iterations = None

    # Get the attack
    attack_function = attacks[attack]

    print("Training the model using adversarial examples...")
    model.train()

    # Use a pretty progress bar to show updates
    for epoch in tnrange(epochs, desc="Adversarial Training Progress"):
        adjust_learning_rate(optimizer, epoch, learning_rate)
        total_epoch_loss = 0

        for _, (images, labels) in enumerate(tqdm(trainSetLoader, desc="Batches")):
            # Cast to proper tensors
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            # Run the attack
            model.eval()
            perturbed_images = attack_function(
                images,
                labels,
                model,
                nn.CrossEntropyLoss(),
                epsilon=epsilon,
                alpha=alpha,
                scale=True,
                iterations=iterations,
            )
            model.train()

            # Predict and optimise
            logits = model(perturbed_images)
            loss = loss_function(logits, labels)

            # Early stopping if loss becomes nan
            if np.isnan(loss.item()):
                print("...terminating early due to loss being nan...")

                return model

            # Track total loss
            total_epoch_loss += loss.item()

            # Gradient descent
            loss.backward()

            # Also clip the gradients (for some reason they super explode and loss becomes nan)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 10)
            optimizer.step()

        # To track if the model is getting better
        print("Loss is ", total_epoch_loss)
        model.eval()
        clean_test.test_trained_model(model, testSetLoader)
        attacking.attack_model(
            model,
            testSetLoader,
            "FGSM",
            attacks["FGSM"],
            epsilon=epsilon,
        )
        model.train()

    print("... done!")

    # Return the trained model
    return model

In [6]:
# Quite similar to the one above, but this one introduces IAT (Interpolated Adversarial Training)
import defenses.iat as iat


def get_interpolated_adversarially_trained_model(attack, **kwargs):
    # Helps adjust learning rate for better results
    def adjust_learning_rate(optimizer, epoch, learning_rate):
        actual_learning_rate = learning_rate
        if epoch >= 15:
            actual_learning_rate = 0.01
        if epoch >= 20:
            actual_learning_rate = 0.001
        for param_group in optimizer.param_groups:
            param_group["lr"] = actual_learning_rate

    # Various training parameters
    epochs = 30
    learning_rate = 0.1

    # Network parameters
    loss_function = nn.CrossEntropyLoss()
    model = resnet.ResNet18()
    model = model.to(device)
    model = nn.DataParallel(model)

    # Consider using ADAM here as another gradient descent algorithm
    optimizer = torch.optim.SGD(
        model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002
    )

    # Check if using epsilon
    if "epsilon" in kwargs:
        epsilon = kwargs["epsilon"]
    else:
        epsilon = None

    # Check if using alpha
    if "alpha" in kwargs:
        alpha = kwargs["alpha"]
    else:
        alpha = None

    # Get iterations
    if "iterations" in kwargs:
        iterations = kwargs["iterations"]
    else:
        iterations = None

    # Get the attack
    attack_function = attacks[attack]

    print("Training the model using adversarial examples...")
    model.train()

    # Use a pretty progress bar to show updates
    for epoch in tnrange(epochs, desc="Adversarial Training Progress"):
        adjust_learning_rate(optimizer, epoch, learning_rate)

        for _, (images, labels) in enumerate(tqdm(trainSetLoader, desc="Batches")):
            # Cast to proper tensors
            images, labels = images.to(device), labels.to(device)

            # Make sure previous step gradients are not used
            optimizer.zero_grad()

            # Use manifold mixup to modify the data
            (
                benign_mix_images,
                benign_mix_labels_a,
                benign_mix_labels_b,
                benign_mix_lamda,
            ) = iat.mix_inputs(1, images, labels)

            # Predict and calculate benign loss
            benign_logits = model(benign_mix_images)
            benign_loss = iat.mixup_loss_function(
                loss_function,
                benign_mix_lamda,
                benign_logits,
                benign_mix_labels_a,
                benign_mix_labels_b,
            )

            # Run the adversarial attack
            model.eval()
            perturbed_images = attack_function(
                images,
                labels,
                model,
                loss_function,
                epsilon=epsilon,
                alpha=alpha,
                scale=True,
                iterations=iterations,
            )
            model.train()

            # Use manifold mixup on the adversarial data
            (
                adversarial_mix_images,
                adversarial_mix_labels_a,
                adversarial_mix_labels_b,
                adversarial_mix_lamda,
            ) = iat.mix_inputs(1, perturbed_images, labels)

            # Predict and calculate adversarial loss
            adversarial_logits = model(adversarial_mix_images)
            adversarial_loss = iat.mixup_loss_function(
                loss_function,
                adversarial_mix_lamda,
                adversarial_logits,
                adversarial_mix_labels_a,
                adversarial_mix_labels_b,
            )

            # Take average of the two losses
            loss = (benign_loss + adversarial_loss) / 2

            # Gradient descent
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 10)
            optimizer.step()

        # To track if the model is getting better
        model.eval()
        clean_test.test_trained_model(model, testSetLoader)
        attacking.attack_model(
            model,
            testSetLoader,
            "FGSM",
            attacks["FGSM"],
            epsilon=epsilon,
        )
        model.train()

    print("... done!")

    # Return the trained model
    return model

### FGSM Adversarial Training

In [7]:
fgsm_model = get_adversarially_trained_model("FGSM", epsilon=0.075)

# From this point on, we simply want to evaluate
fgsm_model.eval()
clean_test.test_trained_model(fgsm_model, testSetLoader)

Training the model using adversarial examples...


Adversarial Training Progress:   0%|          | 0/30 [00:00<?, ?it/s]

Batches:   0%|          | 0/391 [00:00<?, ?it/s]

Loss is  879.6630033254623
Testing the model...


Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

...

... done! Accuracy: 50.24%
Testing the model under FGSM Attack using epsilon = 0.075, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 2.48%


... done!
Testing the model...


Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 50.24%


In [8]:
# Several values to use for the epsilons
epsilons = [0, 0.05, 0.1, 0.2, 0.35, 0.55]

In [9]:
for epsilon in epsilons:
    attacking.attack_model(
        fgsm_model,
        testSetLoader,
        "FGSM",
        attacks["FGSM"],
        epsilon=epsilon,
    )

Testing the model under FGSM Attack using epsilon = 0, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 50.24%


Testing the model under FGSM Attack using epsilon = 0.05, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 3.47%


Testing the model under FGSM Attack using epsilon = 0.1, alpha = None...


...

... done! Accuracy: 4.11%


Testing the model under FGSM Attack using epsilon = 0.35, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 5.02%


Testing the model under FGSM Attack using epsilon = 0.55, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 6.0%




In [10]:
# Several values to use for the epsilons
epsilons = [0, 0.05, 4 / 255, 0.1, 0.3]

In [11]:
for epsilon in epsilons:
    attacking.attack_model(
        fgsm_model,
        testSetLoader,
        "PGD",
        attacks["PGD"],
        epsilon=epsilon,
        alpha=(2 / 255),
        iterations=20,
    )

Testing the model under PGD Attack using epsilon = 0, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 50.24%


Testing the model under PGD Attack using epsilon = 0.05, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 0.76%


Testing the model under PGD Attack using epsilon = 0.01568627450980392, alpha = 0.00784313725490196...


...

... done! Accuracy: 17.91%


Testing the model under PGD Attack using epsilon = 0.1, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 0.0%


Testing the model under PGD Attack using epsilon = 0.3, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 0.0%




In [12]:
# Make sure to test the final accuracy of the model
clean_test.test_trained_model(fgsm_model, testSetLoader)

Testing the model...


Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 50.24%


In [13]:
# Save the FGSM model
torch.save(fgsm_model, "./cifar10_trained_fgsm_model")

### PGD Adversarial Training

In [14]:
pgd_model = get_adversarially_trained_model(
    "PGD", epsilon=(8 / 255), alpha=(2 / 255), iterations=7
)

Training the model using adversarial examples...


Adversarial Training Progress:   0%|          | 0/30 [00:00<?, ?it/s]

Batches:   0%|          | 0/391 [00:00<?, ?it/s]

Loss is  873.844141960144
Testing the model...


Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

...

Loss is  553.3096624612808
Testing the model...


Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 66.44%
Testing the model under FGSM Attack using epsilon = 0.03137254901960784, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 42.37%


... done!


In [15]:
# Several values to use for the epsilons
epsilons = [0, 0.05, 0.1, 0.2, 0.35, 0.55]

In [16]:
for epsilon in epsilons:
    attacking.attack_model(
        pgd_model,
        testSetLoader,
        "FGSM",
        attacks["FGSM"],
        epsilon=epsilon,
    )

Testing the model under FGSM Attack using epsilon = 0, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 72.74%


Testing the model under FGSM Attack using epsilon = 0.05, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 25.43%


Testing the model under FGSM Attack using epsilon = 0.1, alpha = None...


...

... done! Accuracy: 9.97%


Testing the model under FGSM Attack using epsilon = 0.35, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 9.51%


Testing the model under FGSM Attack using epsilon = 0.55, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 11.45%




In [17]:
# Several values to use for the epsilons
epsilons = [0, 0.05, 4 / 255, 0.1, 0.3]

In [18]:
for epsilon in epsilons:
    attacking.attack_model(
        pgd_model,
        testSetLoader,
        "PGD",
        attacks["PGD"],
        epsilon=epsilon,
        alpha=(2 / 255),
        iterations=7,
    )

Testing the model under PGD Attack using epsilon = 0, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 72.74%


Testing the model under PGD Attack using epsilon = 0.05, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 24.77%


Testing the model under PGD Attack using epsilon = 0.01568627450980392, alpha = 0.00784313725490196...


...

... done! Accuracy: 51.67%


Testing the model under PGD Attack using epsilon = 0.1, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 18.51%


Testing the model under PGD Attack using epsilon = 0.3, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 11.21%




In [19]:
# Make sure to test the final accuracy of the model
clean_test.test_trained_model(pgd_model, testSetLoader)

Testing the model...


Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 72.74%


## Loading a good (saved) model

In [20]:
pgd_model = torch.load("./models_data/cifar10_pgd_model_200_epochs")
pgd_model.eval()

clean_test.test_trained_model(pgd_model, testSetLoader)

Testing the model...


Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 83.53%


In [21]:
# Several values to use for the epsilons
epsilons = [0, 0.05, 0.1, 0.2, 0.35, 0.55]

In [22]:
for epsilon in epsilons:
    attacking.attack_model(
        pgd_model,
        testSetLoader,
        "FGSM",
        attacks["FGSM"],
        epsilon=epsilon,
    )

Testing the model under FGSM Attack using epsilon = 0, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 83.53%


Testing the model under FGSM Attack using epsilon = 0.05, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 38.91%


Testing the model under FGSM Attack using epsilon = 0.1, alpha = None...


...

... done! Accuracy: 18.76%


Testing the model under FGSM Attack using epsilon = 0.35, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 18.24%


Testing the model under FGSM Attack using epsilon = 0.55, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 16.7%




In [23]:
# Several values to use for the epsilons
epsilons = [0, 0.05, 4 / 255, 0.1, 0.3]

In [24]:
for epsilon in epsilons:
    attacking.attack_model(
        pgd_model,
        testSetLoader,
        "PGD",
        attacks["PGD"],
        epsilon=epsilon,
        alpha=(2 / 255),
        iterations=7,
    )

Testing the model under PGD Attack using epsilon = 0, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 83.53%


Testing the model under PGD Attack using epsilon = 0.05, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 37.05%


Testing the model under PGD Attack using epsilon = 0.01568627450980392, alpha = 0.00784313725490196...


...

... done! Accuracy: 64.3%


Testing the model under PGD Attack using epsilon = 0.1, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 27.28%


Testing the model under PGD Attack using epsilon = 0.3, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 4.93%




In [25]:
attacking.attack_model(
    pgd_model,
    testSetLoader,
    "PGD",
    attacks["PGD"],
    epsilon=(4 / 255),
    alpha=(2 / 255),
    iterations=20,
)

Testing the model under PGD Attack using epsilon = 0.01568627450980392, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 64.17%




## Interpolated Adversarial Training (IAT)

In [26]:
iat_pgd_model = get_interpolated_adversarially_trained_model(
    "PGD", epsilon=(8 / 255), alpha=(2 / 255), iterations=7
)

Training the model using adversarial examples...


Adversarial Training Progress:   0%|          | 0/30 [00:00<?, ?it/s]

Batches:   0%|          | 0/391 [00:00<?, ?it/s]

Testing the model...


Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

...

Testing the model...


Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 62.28%
Testing the model under FGSM Attack using epsilon = 0.03137254901960784, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 35.33%


... done!


In [27]:
# iat_pgd_model = torch.load("cifar_trained_iat_model")

In [28]:
# Several values to use for the epsilons
epsilons = [0, 0.05, 0.1, 0.2, 0.35, 0.55]

In [29]:
for epsilon in epsilons:
    attacking.attack_model(
        iat_pgd_model,
        testSetLoader,
        "FGSM",
        attacks["FGSM"],
        epsilon=epsilon,
    )

Testing the model under FGSM Attack using epsilon = 0, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 60.2%


Testing the model under FGSM Attack using epsilon = 0.05, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 23.22%


Testing the model under FGSM Attack using epsilon = 0.1, alpha = None...


...

... done! Accuracy: 3.93%


Testing the model under FGSM Attack using epsilon = 0.35, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 2.64%


Testing the model under FGSM Attack using epsilon = 0.55, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 2.32%




In [30]:
# Several values to use for the epsilons
epsilons = [0, 0.05, 4 / 255, 0.1, 0.3]

In [31]:
for epsilon in epsilons:
    attacking.attack_model(
        iat_pgd_model,
        testSetLoader,
        "PGD",
        attacks["PGD"],
        epsilon=epsilon,
        alpha=(2 / 255),
        iterations=7,
    )

Testing the model under PGD Attack using epsilon = 0, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 60.2%


Testing the model under PGD Attack using epsilon = 0.05, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 25.84%


Testing the model under PGD Attack using epsilon = 0.01568627450980392, alpha = 0.00784313725490196...


...

... done! Accuracy: 45.64%


Testing the model under PGD Attack using epsilon = 0.1, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 19.82%


Testing the model under PGD Attack using epsilon = 0.3, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 13.37%




In [32]:
attacking.attack_model(
    iat_pgd_model,
    testSetLoader,
    "PGD",
    attacks["PGD"],
    epsilon=(4 / 255),
    alpha=(2 / 255),
    iterations=20,
)

Testing the model under PGD Attack using epsilon = 0.01568627450980392, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 45.59%




## Interpolated Adversarial Training (with Jacobian Regularization)

In [33]:
from jacobian import JacobianReg

import defenses.iat as iat


def get_jacobian_interpolated_adversarially_trained_model(attack, **kwargs):
    # Helps adjust learning rate for better results
    def adjust_learning_rate(optimizer, epoch, learning_rate):
        actual_learning_rate = learning_rate
        if epoch >= 150:
            actual_learning_rate = 0.01
        if epoch >= 200:
            actual_learning_rate = 0.001
        for param_group in optimizer.param_groups:
            param_group["lr"] = actual_learning_rate

    # Various training parameters
    epochs = 30
    learning_rate = 0.1

    # Network parameters
    loss_function = nn.CrossEntropyLoss()
    model = resnet.ResNet18()
    model = model.to(device)
    model = nn.DataParallel(model)

    # Jacobian regularization
    jacobian_reg = JacobianReg()
    jacobian_reg_lambda = 0.01

    # Consider using ADAM here as another gradient descent algorithm
    optimizer = torch.optim.SGD(
        model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4
    )

    # Check if using epsilon
    if "epsilon" in kwargs:
        epsilon = kwargs["epsilon"]
    else:
        epsilon = None

    # Check if using alpha
    if "alpha" in kwargs:
        alpha = kwargs["alpha"]
    else:
        alpha = None

    # Get iterations
    if "iterations" in kwargs:
        iterations = kwargs["iterations"]
    else:
        iterations = None

    # Get the attack
    attack_function = attacks[attack]

    print("Training the model using adversarial examples...")

    # Use a pretty progress bar to show updates
    for epoch in tnrange(epochs, desc="Adversarial Training Progress"):
        adjust_learning_rate(optimizer, epoch, learning_rate)

        for _, (images, labels) in enumerate(tqdm(trainSetLoader, desc="Batches")):
            # Cast to proper tensors
            images, labels = images.to(device), labels.to(device)

            # Use manifold mixup to modify the data
            (
                benign_mix_images,
                benign_mix_labels_a,
                benign_mix_labels_b,
                benign_mix_lamda,
            ) = iat.mix_inputs(1, images, labels)

            # Run the adversarial attack
            model.eval()
            perturbed_images = attack_function(
                images,
                labels,
                model,
                loss_function,
                epsilon=epsilon,
                alpha=alpha,
                scale=True,
                iterations=iterations,
            )
            model.train()

            # Use manifold mixup on the adversarial data
            (
                adversarial_mix_images,
                adversarial_mix_labels_a,
                adversarial_mix_labels_b,
                adversarial_mix_lamda,
            ) = iat.mix_inputs(1, perturbed_images, labels)

            # Predictions are regularization
            benign_mix_images.requires_grad = True
            adversarial_mix_images.requires_grad = True

            optimizer.zero_grad()

            # Predict and calculate benign loss
            benign_logits = model(benign_mix_images)

            benign_loss = iat.mixup_loss_function(
                loss_function,
                benign_mix_lamda,
                benign_logits,
                benign_mix_labels_a,
                benign_mix_labels_b,
            )

            # Introduce Jacobian regularization
            jacobian_reg_loss = jacobian_reg(benign_mix_images, benign_logits)

            # Total benign loss
            benign_loss = benign_loss + jacobian_reg_lambda * jacobian_reg_loss

            # Predict and calculate adversarial loss
            adversarial_logits = model(adversarial_mix_images)
            adversarial_loss = iat.mixup_loss_function(
                loss_function,
                adversarial_mix_lamda,
                adversarial_logits,
                adversarial_mix_labels_a,
                adversarial_mix_labels_b,
            )

            # Introduce Jacobian regularization
            jacobian_reg_loss = jacobian_reg(adversarial_mix_images, adversarial_logits)

            # Total adversarial loss
            adversarial_loss = (
                adversarial_loss + jacobian_reg_lambda * jacobian_reg_loss
            )

            # Take average of the two losses
            loss = (benign_loss + adversarial_loss) / 2

            # Gradient descent
            loss.backward()
            optimizer.step()

    print("... done!")

    # Return the trained model
    return model

In [34]:
iat_pgd_model = get_interpolated_adversarially_trained_model(
    "PGD", epsilon=0.6, alpha=(2 / 255), iterations=7
)

iat_pgd_model.eval()

Training the model using adversarial examples...


Adversarial Training Progress:   0%|          | 0/30 [00:00<?, ?it/s]

Batches:   0%|          | 0/391 [00:00<?, ?it/s]

Testing the model...


Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

...

... done! Accuracy: 10.0%
Testing the model under FGSM Attack using epsilon = 0.6, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 10.0%




Batches:   0%|          | 0/391 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Several values to use for the epsilons
epsilons = [0, 0.05, 0.1, 0.2, 0.35, 0.55]

In [ ]:
# Run test for each epsilon
for epsilon in epsilons:
    attacking.attack_model(
        iat_pgd_model,
        testSetLoader,
        "FGSM",
        attacks["FGSM"],
        epsilon=epsilon,
    )

In [ ]:
# Several values to use for the epsilons
epsilons = [0, 0.05, 4 / 255, 0.1, 0.3]

In [ ]:
for epsilon in epsilons:
    attacking.attack_model(
        iat_pgd_model,
        testSetLoader,
        "PGD",
        attacks["PGD"],
        epsilon=epsilon,
        alpha=(2 / 255),
        iterations=7,
    )

In [ ]:
clean_test.test_trained_model(iat_pgd_model, testSetLoader)